New notebook to test the difference between the tech rec mongo db and gal alum.
Currently, code working for gal alum but not tech recruiters.
1. Copy progress from mongodb_test
2. Replicate the gal alum process from
* Connection to mongo db
* Pull data into df
* Manipulate to correct format
* Create csv of all companies
* Create csvs of individual companies

3. New notebook section for tech recruiters

In [1]:
import pandas as pd
import mongo
# import est

# Galvanize Alumni CSV build

In [2]:
mongo.connect_mongo()
mongo.connect_coll('gal_part_proj', 'gal_alum')
cursor = mongo.coll.find({}, {'_id':0})

In [3]:
df = pd.DataFrame(cursor)

In [4]:
df = df.T
df.head()

,0,1,2,3,4,5,6,7,8,9,...,457,458,459,460,461,462,463,464,465,466
University of Texas at Austin,People Link 404,People Link 404,People Link 404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apple,NaN,NaN,NaN,{'Erin Gong': 'https://www.linkedin.com/in/eri...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dell Technologies,NaN,NaN,NaN,NaN,{'Daniel Donovan': 'https://www.linkedin.com/i...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IBM,NaN,NaN,NaN,NaN,NaN,{'James Helfrich': 'https://www.linkedin.com/i...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
National Instruments,NaN,NaN,NaN,NaN,NaN,NaN,No results,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
index = list()
results = list()

for idx, row in df.iterrows():
    index.append(idx)
    results.append(row[row.notnull()].values[0])

clean_df = pd.DataFrame(results, index=index)
clean_df.rename({0: 'Scrape Results'}, axis=1, inplace=True)
clean_df.head()

,Scrape Results
University of Texas at Austin,People Link 404
Apple,{'Erin Gong': 'https://www.linkedin.com/in/eri...
Dell Technologies,{'Daniel Donovan': 'https://www.linkedin.com/i...
IBM,{'James Helfrich': 'https://www.linkedin.com/i...
National Instruments,No results


In [52]:
type(results[0])

str

In [53]:
type(results[1])

dict

# Austin Technical Recruiters CSV build

In [10]:
mongo.connect_mongo()
mongo.connect_coll('gal_part_proj', 'ATX_tech_rec')
cursor = mongo.coll.find({}, {'_id':0})

df_tr = pd.DataFrame(cursor)
df_tr = df_tr.T

## Dataframe for Austin Tech Recruiters

In [9]:
df_tr.head()

,0,1,2,3,4,5,6,7,8,9,...,189,190,191,192,193,194,195,196,197,198
9Gauge Partners,{'Carrie Guillory': 'https://www.linkedin.com/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Abbott,NaN,{'Brandy Williams Broden': 'https://www.linked...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aceable,NaN,NaN,"{'Andrew Bolisay (he, him, his)': 'https://www...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AdbaKx,NaN,NaN,NaN,Company Page 404,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Addison Group,NaN,NaN,NaN,NaN,{'Hattie Cox': 'https://www.linkedin.com/in/ha...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Dataframe for Galvanize Alumni

In [13]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,457,458,459,460,461,462,463,464,465,466
University of Texas at Austin,People Link 404,People Link 404,People Link 404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apple,NaN,NaN,NaN,{'Erin Gong': 'https://www.linkedin.com/in/eri...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dell Technologies,NaN,NaN,NaN,NaN,{'Daniel Donovan': 'https://www.linkedin.com/i...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IBM,NaN,NaN,NaN,NaN,NaN,{'James Helfrich': 'https://www.linkedin.com/i...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
National Instruments,NaN,NaN,NaN,NaN,NaN,NaN,No results,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Austin Tech Recruiters pipeline error corrections
My dataframe pipeline is failing for the tech recruiters mongo pull. Successively passing the Alumni pull with the same process. Steps to troubleshoot proceed.

In [11]:
# GALVANIZE ALUMNI DATAFRAME
clean_df.head()

,Scrape Results
University of Texas at Austin,People Link 404
Apple,{'Erin Gong': 'https://www.linkedin.com/in/eri...
Dell Technologies,{'Daniel Donovan': 'https://www.linkedin.com/i...
IBM,{'James Helfrich': 'https://www.linkedin.com/i...
National Instruments,No results


In [62]:
# pipeline for ATX Tech Recruiters
index_tr = list()
results_tr = list()

for idx, row in df_tr.iterrows():
    index_tr.append(idx)
    results_tr.append(row[row.notnull()].values[0])


In [63]:
clean_df_tr = pd.DataFrame(results_tr, index=index_tr)
# clean_df_tr.rename({0: 'Scrape Results'}, axis=1, inplace=True)

AttributeError: 'str' object has no attribute 'keys'

In [57]:
type(results)

list

In [60]:
results

['People Link 404',
 {'Erin Gong': 'https://www.linkedin.com/in/eringong/',
  'Kendal Holcombe (Chilcott)': 'https://www.linkedin.com/in/kendal-holcombe/',
  'Ryan Hunter': 'https://www.linkedin.com/in/ryan-hunter-1313a53b/',
  'Bob Mickus': 'https://www.linkedin.com/in/bobmickus/',
  'Isaac Lessard': 'https://www.linkedin.com/in/isaaclessard/',
  'Zhifan (Jeff) Sang': 'https://www.linkedin.com/in/zfsang/',
  'Sierra Murphy': 'https://www.linkedin.com/in/sierra-murphy-81b75ba2/',
  'Hao Ding': 'https://www.linkedin.com/in/haoding1/',
  'Udaiveer Singh': 'https://www.linkedin.com/in/udaiveers/',
  'Kevin Becerra': 'https://www.linkedin.com/in/kevinbece55/',
  'Charlie (Changsong) Ding': 'https://www.linkedin.com/in/changsongding/',
  'Bryce Schmidtchen': 'https://www.linkedin.com/in/bryceschmidtchen/',
  'Omar Sobh': 'https://www.linkedin.com/in/omarsobh/',
  'Wolfgang Morton': 'https://www.linkedin.com/in/wolfgangmorton/',
  'DeBronis Warren': 'https://www.linkedin.com/in/debronis/',
 

In [19]:
# ATX TECH REC DATAFRAME
clean_df_tr.head()

,Carrie Guillory
9Gauge Partners,https://www.linkedin.com/in/carrieguillory/


In [22]:
_9gauge = df_tr.loc['9Gauge Partners']

In [38]:
abbott = df_tr.loc['Abbott']

In [43]:
abbott[abbott.notnull()].values[0]

{'Brandy Williams Broden': 'https://www.linkedin.com/in/brandy-williams-broden-0463508/',
 'Lindsey M': 'https://www.linkedin.com/in/lindseym3/',
 'Kim Imbrogno-Maloney': 'https://www.linkedin.com/in/kim-imbrogno-maloney-676a238/',
 'Julie Decker': 'https://www.linkedin.com/in/juliedecker/',
 'Dona Goswami': 'https://www.linkedin.com/in/donagoswami/',
 'Darin Bertram': 'https://www.linkedin.com/in/darinbertram/',
 'Latha Krishnakumar': 'https://www.linkedin.com/in/lathakrishnakumar/',
 'Jennifer Morgan': 'https://www.linkedin.com/in/jenniferroark/',
 'Kate Chapa': 'https://www.linkedin.com/in/kate-chapa-462557b8/',
 'Katie Korrison': 'https://www.linkedin.com/in/katiekorrison/',
 '👋🏼 Cindy Puwarie': 'https://www.linkedin.com/in/cindypuwarie/',
 'Shirley F Womack, CIR': 'https://www.linkedin.com/in/sfrances/',
 'Julian Gardner': 'https://www.linkedin.com/in/julian-gardner-b265551/',
 'Brittane Russell': 'https://www.linkedin.com/in/brittane-russell/',
 'Anna Smith': 'https://www.linkedi

In [45]:
pd.DataFrame(abbott[abbott.notnull()].values[0], index=['Abbott'])

,Brandy Williams Broden,Lindsey M,Kim Imbrogno-Maloney,Julie Decker,Dona Goswami,Darin Bertram,Latha Krishnakumar,Jennifer Morgan,Kate Chapa,Katie Korrison,...,Samantha Mari,Meltem Cakir,Kuburat (KB) Badejo,Jose Tolentino,Jessica McDaniel,Doris Smart,Matt Kerr,Karen Kremer,"Brenda Hernandez, MBA",Brianna Purser
Abbott,https://www.linkedin.com/in/brandy-williams-br...,https://www.linkedin.com/in/lindseym3/,https://www.linkedin.com/in/kim-imbrogno-malon...,https://www.linkedin.com/in/juliedecker/,https://www.linkedin.com/in/donagoswami/,https://www.linkedin.com/in/darinbertram/,https://www.linkedin.com/in/lathakrishnakumar/,https://www.linkedin.com/in/jenniferroark/,https://www.linkedin.com/in/kate-chapa-462557b8/,https://www.linkedin.com/in/katiekorrison/,...,https://www.linkedin.com/in/smprada/,https://www.linkedin.com/in/meltem-cakir-5680464/,https://www.linkedin.com/in/kuburat-badejo/,https://www.linkedin.com/in/jose-tolentino-108...,https://www.linkedin.com/in/recruiterjessicamo...,https://www.linkedin.com/in/doriszavala/,https://www.linkedin.com/in/mattkerrhealthcare/,https://www.linkedin.com/in/karen-kremer-6b9949/,https://www.linkedin.com/in/brenda-hernandez-hr/,https://www.linkedin.com/in/brianna-purser-439...


In [46]:
alum_abbott = df.loc['Abbott']

In [49]:
pd.DataFrame(alum_abbott[alum_abbott.notnull()].values[0], index=['Abbott'])

,Diogo C Novaes
Abbott,https://www.linkedin.com/in/diogo-c-novaes/


In [27]:
_9gauge[_9gauge.notnull()].values[0]

{'Carrie Guillory': 'https://www.linkedin.com/in/carrieguillory/'}

## start by looking at tech recruiter `results` & `index` loop construction

In [7]:
cnt = 0
for idx, row in df_tr.iterrows():
#     index.append(idx)
    print(type(row[row.notnull()].values[0]))
    cnt +=1
    if cnt == 4:
        break
    

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'str'>


In [8]:
cnt = 0
for idx, row in df.iterrows():
#     index.append(idx)
    print(type(row[row.notnull()].values[0]))
    cnt +=1
    if cnt == 4:
        break

<class 'str'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
